In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [2]:
# Read the .csv file
df_mazda = pd.read_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\\raw_datasets\mazdaR.csv")

<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_4048\1367882422.py:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
  df_mazda = pd.read_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\\raw_datasets\mazdaR.csv")


In [3]:
# Check the df info
df_mazda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4484 entries, 0 to 4483
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ModType      3701 non-null   object
 1   HARDWARE     3701 non-null   object
 2   18881 P/N    2030 non-null   object
 3   Catch        3100 non-null   object
 4   12A650       2578 non-null   object
 5   EPC P/N      2284 non-null   object
 6   Mazda Model  3695 non-null   object
 7   Year         3671 non-null   object
 8   Engine       3637 non-null   object
 9   Trans        1785 non-null   object
 10  Emissions    1911 non-null   object
 11  SecType      1841 non-null   object
 12  Key          12 non-null     object
 13  Description  2777 non-null   object
 14  VIN EXAMPLE  337 non-null    object
 15  Unnamed: 15  50 non-null     object
dtypes: object(16)
memory usage: 560.6+ KB


### Delete all columns and keep only the part number related ones:
- ModType
- HARDWARE
- 18881 P/N
- 12A650
- EPC P/N

In [4]:
# Remove rows where all cols have null (NaN) values,
# keeping only rows with at least one value (non-null) entry.
df_mazda_wo_nulls = df_mazda[~df_mazda.isnull().all(axis=1)]

In [5]:
# Keep only rows where the cols have > 1 non-null values.
# The goal is to remove the lines where only col "18881 P/N" is filled out with a comment.
df_mazda_rm_one_line_notNull = df_mazda_wo_nulls[df_mazda_wo_nulls.notnull().sum(axis=1) != 1]

In [6]:
# Index list of rows filled out with the columns
index_list_repeated_cols = df_mazda_rm_one_line_notNull[df_mazda_rm_one_line_notNull['ModType'] == 'ModType'].index

In [7]:
# Drop rows with col names
df_mazda_dropped_cols = df_mazda_rm_one_line_notNull.drop(index=index_list_repeated_cols)

In [8]:
# Function to fill out null rows
def fill_null_rows(df):
    # Loop to iterate over all cols
    for col in df.columns:
        df[col] = df[col].fillna("Not Available")
    return df

In [9]:
# Function to remove the whitespaces
def remove_whitespaces(df):
    # Loop to iterate over all cols
    for col in df.columns:
        # Remove the writespaces
        df[col] = df[col].str.strip()

    return df

In [10]:
# Function to split multiple part numbers originally in the same line to one row each
def split_part_numbers(df, col):

    # Call the function to remove the whitespaces
    df_rm_spaces = remove_whitespaces(df)

    df_copy = df_rm_spaces.copy()

    # Create a list with the items in each row 
    df_copy[col] = df_copy[col].str.split(",")
    # Explode the items in different rows each and keep the info from the other rows
    df_splitted = df_copy.explode(col)
    # Return the df with the exploded part numbers
    return df_splitted

In [11]:
# Show the cols
df_mazda_dropped_cols.columns

Index(['ModType', 'HARDWARE', '18881 P/N', 'Catch', '12A650', 'EPC P/N',
       'Mazda Model', 'Year', 'Engine', 'Trans', 'Emissions', 'SecType', 'Key',
       'Description', 'VIN EXAMPLE', 'Unnamed: 15'],
      dtype='object')

In [12]:
# List with cols to keep
cols_to_keep = ['HARDWARE', 'ModType', '18881 P/N', '12A650', 'EPC P/N']

In [13]:
# Create a df with needed cols
df_mazda_filtered_cols = df_mazda_dropped_cols[cols_to_keep]

In [14]:
# Call the function to fill out null values
df_mazda_fill_nulls = fill_null_rows(df_mazda_filtered_cols)

C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_4048\132036243.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna("Not Available")


In [15]:
# Map to rename Hardware col
map_dict = {
    "HARDWARE": "hardware"
}

In [16]:
# Rename hdw col
df_mazda_rn_hdw_col = df_mazda_fill_nulls.rename(columns=map_dict)

In [17]:
# Show the cols
df_mazda_rn_hdw_col.columns

Index(['hardware', 'ModType', '18881 P/N', '12A650', 'EPC P/N'], dtype='object')

### Call the function to the explode all items of the cols

In [18]:
df_mazda_expl_hdw_col = split_part_numbers(df_mazda_rn_hdw_col, "hardware")

In [19]:
df_mazda_expl_modtype_col = split_part_numbers(df_mazda_expl_hdw_col, "ModType")

In [20]:
df_mazda_expl_18881Pn_col = split_part_numbers(df_mazda_expl_modtype_col, "18881 P/N")

In [21]:
df_mazda_expl_12A650_col = split_part_numbers(df_mazda_expl_18881Pn_col, "12A650")

In [22]:
df_mazda_expl_epcPn_col = split_part_numbers(df_mazda_expl_12A650_col, "EPC P/N")

In [23]:
# Reset index
df_mazda_reset_index = df_mazda_expl_epcPn_col.reset_index(drop=True)

## Standardizing Part Number Dataset Columns
Each part number dataset contains columns with different names because they originate from different manufacturers. One of the goals of cleaning these datasets is to create a standard format that allows for the identification of the hardware part number when inputting other part numbers, such as software part numbers or system part numbers.

Since the identification of the hardware part number will be performed by inputting multiple part numbers at once (via a .csv file), it is necessary — except for the hardware column — to rename the other columns to a standard format: `pnNum` (e.g., `pn1`, `pn2`, etc.). This ensures consistency and enables automated matching and processing across datasets from various manufacturers.

In [24]:
def rename_col_names(df):
    '''
    Except the column hardware, the function standardize the names for pnNumber (pn1, pn2).
    Needed due to each part number dataset has different column names, so that 

    Parameters:
    df (DataFrame): main df.
    col_list (list): List of original column names.

    Returns
    df: Return the main df with the renamed cols

    '''

    # List with the col names
    col_name_list = list(df.columns) 
    
    # Dict to append the old names (key): new names (values) 
    new_cols_dict = {}

    # Count the items under the col list
    col_num = len(col_name_list)
    # Loop to iterate over the list
    for col in col_name_list:
        # Condition to define when lower the col name and when rename it
        if col == "hardware":
            new_cols_dict[col] = col.lower()
        else:
            # Convert the string to number to sum with the stirng pn and
            # append as value to the dict
            new_cols_dict[col] = "pn" + str(col_num)
            # Return the number from string to int
            col_num = int(col_num)
        
        # Subtract one to create the next col name
        num_col = col_num - 1
        col_num = num_col

    # Return the main df with the cols remanes
    return df.rename(columns=new_cols_dict)

In [25]:
# Call the function to rename the cols
df_mazda_rn_cols = rename_col_names(df_mazda_reset_index)

In [26]:
# Check df info
df_mazda_rn_cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4544 entries, 0 to 4543
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   hardware  4544 non-null   object
 1   pn4       4544 non-null   object
 2   pn3       4544 non-null   object
 3   pn2       4544 non-null   object
 4   pn1       4544 non-null   object
dtypes: object(5)
memory usage: 177.6+ KB


In [30]:
# Export to .csv file
df_mazda_rn_cols.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\data_cleaned\pn-mazda.csv", index=False)

<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_7996\441108231.py:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
  df_mazda_rn_cols.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\data_cleaned\pn-mazda.csv", index=False)


In [27]:
def import_datasets_to_db(df, df_name=str):
    
    '''
    Import the df cleaned to the db on postgreSQL

    Parameters: 
        df (DataFrame): main df to import to the postgreSQL db.
        df_name: string to label the df into the db.

    Returns: 
        Import the df to the db.
    '''
    # Setting up the connection with the PostgreSQL
    dbname="prescreen_diag_data_api"
    user="postgres"
    password="shakey-10"
    host="localhost"
    port="5432"

    # String connection for SQLAlchemy (using psycopg2 as driver)
    engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}")

    return df.to_sql(df_name, engine, if_exists='replace', index=False)

In [28]:
# Call the function to import the df cleaned to the postgreSQLdb
import_datasets_to_db(df_mazda_rn_cols, 'part_numbers_mazda')

544